# dev_phase测试

In [14]:
import torch
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)
use_cuda = torch.cuda.is_available()

Sun Nov  9 20:27:13 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 576.02                 Driver Version: 576.02         CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3090      WDDM  |   00000000:01:00.0  On |                  N/A |
|  0%   39C    P8             29W /  350W |   23727MiB /  24576MiB |     10%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [15]:
SEED = 42
WEIGHTED = True

In [17]:
from datasets import *
import pandas as pd
test = pd.DataFrame(load_dataset("dev_phase/subtask2/dev/spa", split="train"))

Generating train split: 165 examples [00:00, 14996.86 examples/s]


In [18]:
labels = ["political", "racial/ethnic", "religious", "gender/sexual", "other"]
def get_text_and_label(df):  
  return df.rename(columns={"text": "text"})[["id", "text"]]
test = get_text_and_label(test)
print("Test split size:", len(test.index))
test.head()

Test split size: 165


,id,text
0,spa_330f5a37bfec2146ef834b8c3a505888,"""puta weon, los progres arruinaron el indio pi..."
1,spa_09443002d51a05f05c08bde37cb786bc,#mexico | el gobierno informo que esta dialoga...
2,spa_6344425e0525930c1bbf6b37ba0b5ebe,a abortar el mojon que traen atorado
3,spa_b6aa7a3bead18c0b34622e88aabd5928,a comerrrrr\n(la salsa es curry indio)
4,spa_e25a2f619a1dd1c504d5bc44db426065,aborto libre y retroactivo


In [19]:
# EXP = "lr-2e-05-optimizer-AdamW-epochs-10-RMSEduringTraining"
EXP = "09-11-2025-00-03"
# TRANSFORMERS = "top_transformers"  
TRANSFORMERS = "09-11-2025-00-03" 

In [21]:
import os
PATH = "./outputs/"
EXP_PATH = os.path.join(PATH, EXP)
TRANSFORMERS_PATH = os.path.join(PATH, TRANSFORMERS)
print("Current working dir:", EXP_PATH)
OUTPUT = os.path.join(EXP_PATH, "test")
# 第一次使用时创建文件夹
# os.mkdir(EXP_PATH)
# os.mkdir(OUTPUT)

Current working dir: ./outputs/09-11-2025-00-03


In [22]:
models = {    
"mbert-cased": {        
"model_type": "bert",        
"model_name": "models--google-bert--bert-base-multilingual-cased"    
},    
"mbert-uncased": {        
"model_type": "bert",        
"model_name": "models--google-bert--bert-base-multilingual-uncased"    
},    
"roberta": {        
"model_type": "roberta",        
"model_name": "models--FacebookAI--roberta-base"    
},    
"beto-cased": {        
"model_type": "bert",        
"model_name": "models--dccuchile--bert-base-spanish-wwm-cased"    
},    
"beto-uncased": {        
"model_type": "bert",        
"model_name": "models--dccuchile--bert-base-spanish-wwm-uncased"    
},    
"distilbert-multi": {        
"model_type": "distilbert",        
"model_name": "models--distilbert--distilbert-base-multilingual-cased"    
}
}

In [23]:
from simpletransformers.classification import MultiLabelClassificationModel, MultiLabelClassificationArgs
for model, fields in models.items():    
  print("model:",model)
  models[model] = MultiLabelClassificationModel(fields["model_type"], os.path.join(TRANSFORMERS_PATH, model), use_cuda=False)

model: mbert-cased
model: mbert-uncased
model: roberta
model: beto-cased
model: beto-uncased
model: distilbert-multi


In [24]:
import json
best_ensemble = {}
with open(os.path.join(EXP_PATH, "best-ensemble.json")) as json_file:
    best_ensemble = json.load(json_file)
print("----- BEST ENSEMBLE -----")
for field in ["name", "models", "metrics"]:
  print(f"{field}:", best_ensemble.get(field))

----- BEST ENSEMBLE -----
name: ensemble18
models: ['beto-cased', 'beto-uncased']
metrics: {'accuracy': 0.97035, 'macro_f1': 0.98024, 'macro_precision': 0.99442, 'macro_recall': 0.96671, 'weighted_f1': 0.98177}


In [25]:
model_evaluation = {}
for model in best_ensemble.get("models"):  
  with open(os.path.join(os.path.join(EXP_PATH, model), "model-evaluation.json")) as json_file:
      model_evaluation[model] = json.load(json_file)

In [26]:
from sklearn.preprocessing import normalize
def vote(predictions, weighted=False, weights=None):
  votes = list()  
  for i in range(len(labels)):    
    curr_label_preds = [preds[i] for preds in predictions]    
    voting = sum(curr_label_preds * weights) if weighted else sum(curr_label_preds)/len(curr_label_preds)
    votes.append(0 if voting < 0.5 else 1)  
  return votes
test_predictions = list()
def predict_ensemble(ensemble_name, dataset_name, dataset, weighted=False):  
 for i in range(len(dataset.index)):
    predictions = list()
    ensemble_models = best_ensemble.get("models")    
    for model_name in ensemble_models:
      curr_model_outputs = model_evaluation[model_name].get(f"{dataset_name}_model_outputs")
      predictions.append(curr_model_outputs[i])
    weights = list()    
    if weighted:      
      f1_scores_list = [model_evaluation[model_name]["metrics"].get("weighted_f1")                        
                        for model_name in best_ensemble.get("models")]
      weights = normalize([f1_scores_list], norm="l1")[0]
    ensemble_pred = vote(predictions, weighted, weights)
    test_predictions.append(ensemble_pred)

In [27]:
for model_name in best_ensemble.get("models"):
  model_predictions, model_raw_outputs = models.get(model_name).predict(test["text"].tolist())
  model_evaluation[model_name]["test_model_outputs"] = model_raw_outputs
  model_evaluation[model_name]["test_predictions"] = model_predictions
predict_ensemble(best_ensemble.get("name"), "test", test, weighted=WEIGHTED)
n = 5
print(f"First {n} predictions:", test_predictions[:n])

1it [00:02,  2.42s/it]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:12<00:00,  1.68it/s]
1it [00:02,  2.35s/it]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:12<00:00,  1.68it/s]

First 5 predictions: [[0, 1, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]


In [28]:
test_output = test[["id"]].rename(columns={"id": "id"})
for i in range(len(labels)):
  test_output[labels[i]] = [pred[i] for pred in test_predictions]
test_output.head(10)

,id,political,racial/ethnic,religious,gender/sexual,other
0,spa_330f5a37bfec2146ef834b8c3a505888,0,1,0,0,0
1,spa_09443002d51a05f05c08bde37cb786bc,0,0,0,0,0
2,spa_6344425e0525930c1bbf6b37ba0b5ebe,0,0,0,0,1
3,spa_b6aa7a3bead18c0b34622e88aabd5928,0,0,0,0,0
4,spa_e25a2f619a1dd1c504d5bc44db426065,0,0,0,0,0
5,spa_ae37ea460c69447a6e03c3dedf946126,0,0,0,0,0
6,spa_dda5476589aee9d32a5ee7432805999c,1,0,0,0,1
7,spa_1a721702be59d86b876810539ad9cf66,0,0,0,0,0
8,spa_bec4c459366c1a56819a55c01dc1a77d,0,0,0,0,0
9,spa_0a38d1ef4f1ea18eaa9a6d6f5bbfe4b9,1,0,0,0,0


In [29]:
test_output.to_csv(os.path.join(OUTPUT, "results_test.csv"), index=False)

# dev_phase2 测试

In [60]:
import torch
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)
use_cuda = torch.cuda.is_available()

Sun Nov  9 20:38:05 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 576.02                 Driver Version: 576.02         CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3090      WDDM  |   00000000:01:00.0  On |                  N/A |
|  0%   39C    P8             30W /  350W |   23738MiB /  24576MiB |     11%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [61]:
SEED = 42
WEIGHTED = True

In [62]:
from datasets import *
import pandas as pd
test = pd.DataFrame(load_dataset("dev_phase2/subtask2/dev/spa", split="train"))

In [63]:
labels = ["political", "racial/ethnic", "religious", "gender/sexual", "other"]
def get_text_and_label(df):  
  return df.rename(columns={"text": "text"})[["id", "text"]]
test = get_text_and_label(test)
print("Test split size:", len(test.index))
test.head()

Test split size: 165


,id,text
0,spa_330f5a37bfec2146ef834b8c3a505888,"""puta weon, los progres arruinaron el indio pi..."
1,spa_09443002d51a05f05c08bde37cb786bc,#mexico | el gobierno informo que esta dialoga...
2,spa_6344425e0525930c1bbf6b37ba0b5ebe,a abortar el mojon que traen atorado
3,spa_b6aa7a3bead18c0b34622e88aabd5928,a comerrrrr\n(la salsa es curry indio)
4,spa_e25a2f619a1dd1c504d5bc44db426065,aborto libre y retroactivo


In [64]:
# EXP = "lr-2e-05-optimizer-AdamW-epochs-10-RMSEduringTraining"
EXP = "09-11-2025-00-03"
# TRANSFORMERS = "top_transformers"  
TRANSFORMERS = "09-11-2025-00-03" 

In [65]:
import os
PATH = "./outputs/"
EXP_PATH = os.path.join(PATH, EXP)
TRANSFORMERS_PATH = os.path.join(PATH, TRANSFORMERS)
print("Current working dir:", EXP_PATH)
OUTPUT = os.path.join(EXP_PATH, "test")
# 第一次使用时创建文件夹
# os.mkdir(EXP_PATH)
# os.mkdir(OUTPUT)

Current working dir: ./outputs/09-11-2025-00-03


In [66]:
models = {    
"mbert-cased": {        
"model_type": "bert",        
"model_name": "models--google-bert--bert-base-multilingual-cased"    
},    
"mbert-uncased": {        
"model_type": "bert",        
"model_name": "models--google-bert--bert-base-multilingual-uncased"    
},    
"roberta": {        
"model_type": "roberta",        
"model_name": "models--FacebookAI--roberta-base"    
},    
"beto-cased": {        
"model_type": "bert",        
"model_name": "models--dccuchile--bert-base-spanish-wwm-cased"    
},    
"beto-uncased": {        
"model_type": "bert",        
"model_name": "models--dccuchile--bert-base-spanish-wwm-uncased"    
},    
"distilbert-multi": {        
"model_type": "distilbert",        
"model_name": "models--distilbert--distilbert-base-multilingual-cased"    
}
}

In [67]:
from simpletransformers.classification import MultiLabelClassificationModel, MultiLabelClassificationArgs
for model, fields in models.items():    
  print("model:",model)
  models[model] = MultiLabelClassificationModel(fields["model_type"], os.path.join(TRANSFORMERS_PATH, model), use_cuda=False)

model: mbert-cased
model: mbert-uncased
model: roberta
model: beto-cased
model: beto-uncased
model: distilbert-multi


In [68]:
import json
best_ensemble = {}
with open(os.path.join(EXP_PATH, "best-ensemble.json")) as json_file:
    best_ensemble = json.load(json_file)
print("----- BEST ENSEMBLE -----")
for field in ["name", "models", "metrics"]:
  print(f"{field}:", best_ensemble.get(field))

----- BEST ENSEMBLE -----
name: ensemble18
models: ['beto-cased', 'beto-uncased']
metrics: {'accuracy': 0.97035, 'macro_f1': 0.98024, 'macro_precision': 0.99442, 'macro_recall': 0.96671, 'weighted_f1': 0.98177}


In [69]:
model_evaluation = {}
for model in best_ensemble.get("models"):  
  with open(os.path.join(os.path.join(EXP_PATH, model), "model-evaluation.json")) as json_file:
      model_evaluation[model] = json.load(json_file)

In [70]:
from sklearn.preprocessing import normalize
def vote(predictions, weighted=False, weights=None):
  votes = list()  
  for i in range(len(labels)):    
    curr_label_preds = [preds[i] for preds in predictions]    
    voting = sum(curr_label_preds * weights) if weighted else sum(curr_label_preds)/len(curr_label_preds)
    votes.append(0 if voting < 0.5 else 1)  
  return votes
test_predictions = list()
def predict_ensemble(ensemble_name, dataset_name, dataset, weighted=False):  
 for i in range(len(dataset.index)):
    predictions = list()
    ensemble_models = best_ensemble.get("models")    
    for model_name in ensemble_models:
      curr_model_outputs = model_evaluation[model_name].get(f"{dataset_name}_model_outputs")
      predictions.append(curr_model_outputs[i])
    weights = list()    
    if weighted:      
      f1_scores_list = [model_evaluation[model_name]["metrics"].get("weighted_f1")                        
                        for model_name in best_ensemble.get("models")]
      weights = normalize([f1_scores_list], norm="l1")[0]
    ensemble_pred = vote(predictions, weighted, weights)
    test_predictions.append(ensemble_pred)

In [71]:
for model_name in best_ensemble.get("models"):
  model_predictions, model_raw_outputs = models.get(model_name).predict(test["text"].tolist())
  model_evaluation[model_name]["test_model_outputs"] = model_raw_outputs
  model_evaluation[model_name]["test_predictions"] = model_predictions
predict_ensemble(best_ensemble.get("name"), "test", test, weighted=WEIGHTED)
n = 5
print(f"First {n} predictions:", test_predictions[:n])

1it [00:02,  2.43s/it]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:13<00:00,  1.60it/s]
1it [00:02,  2.46s/it]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:13<00:00,  1.60it/s]

First 5 predictions: [[0, 1, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]


In [72]:
test_output = test[["id"]].rename(columns={"id": "id"})
for i in range(len(labels)):
  test_output[labels[i]] = [pred[i] for pred in test_predictions]
test_output.head(10)

,id,political,racial/ethnic,religious,gender/sexual,other
0,spa_330f5a37bfec2146ef834b8c3a505888,0,1,0,0,0
1,spa_09443002d51a05f05c08bde37cb786bc,0,0,0,0,0
2,spa_6344425e0525930c1bbf6b37ba0b5ebe,0,0,0,0,1
3,spa_b6aa7a3bead18c0b34622e88aabd5928,0,0,0,0,0
4,spa_e25a2f619a1dd1c504d5bc44db426065,0,0,0,0,0
5,spa_ae37ea460c69447a6e03c3dedf946126,0,0,0,0,0
6,spa_dda5476589aee9d32a5ee7432805999c,1,0,0,0,1
7,spa_1a721702be59d86b876810539ad9cf66,0,0,0,0,0
8,spa_bec4c459366c1a56819a55c01dc1a77d,0,0,0,0,0
9,spa_0a38d1ef4f1ea18eaa9a6d6f5bbfe4b9,1,0,0,0,0


In [73]:
test_output.to_csv(os.path.join(OUTPUT, "results_test2.csv"), index=False)

# test_phase 测试

In [1]:
import torch
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)
use_cuda = torch.cuda.is_available()

Sun Feb  1 12:26:29 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 576.40                 Driver Version: 576.40         CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3090      WDDM  |   00000000:01:00.0  On |                  N/A |
|  0%   47C    P8             29W /  390W |   23724MiB /  24576MiB |     10%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
SEED = 42
WEIGHTED = True

In [3]:
from datasets import *
import pandas as pd
test = pd.DataFrame(load_dataset("test_phase/subtask2/test/spa", split="train"))

C:\Users\8888\Anaconda3\envs\SemEval2025_Task11\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
labels = ["political", "racial/ethnic", "religious", "gender/sexual", "other"]
def get_text_and_label(df):  
  return df.rename(columns={"text": "text"})[["id", "text"]]
test = get_text_and_label(test)
print("Test split size:", len(test.index))
test.head()

Test split size: 1488


,id,text
0,spa_1927fe24b83567b6383c90c35473cc32,"""penas severas para el aborto porque yo estuve..."
1,spa_73b413c77432d415f2b63397ccbcda28,"""si eres pobre debes abortar"" eso lei"
2,spa_5f0a632b6b0b4544b5fed6c84bd1b997,"#brasil exige a #eeuu ""respeto"" para repatriad..."
3,spa_b23cc0db2a4e552a93fbbfe4e4d2f45f,#honduras | el canciller anuncio la llegada de...
4,spa_90240d10f79a2761cbea0b76e1553a6f,"#internacionales | el presidente de colombia, ..."


In [5]:
# EXP = "lr-2e-05-optimizer-AdamW-epochs-10-RMSEduringTraining"
EXP = "09-11-2025-00-03"
# TRANSFORMERS = "top_transformers"  
TRANSFORMERS = "09-11-2025-00-03" 

In [6]:
import os
PATH = "./outputs/"
EXP_PATH = os.path.join(PATH, EXP)
TRANSFORMERS_PATH = os.path.join(PATH, TRANSFORMERS)
print("Current working dir:", EXP_PATH)
OUTPUT = os.path.join(EXP_PATH, "test")
# 第一次使用时创建文件夹
# os.mkdir(EXP_PATH)
# os.mkdir(OUTPUT)

Current working dir: ./outputs/09-11-2025-00-03


In [7]:
models = {    
"mbert-cased": {        
"model_type": "bert",        
"model_name": "models--google-bert--bert-base-multilingual-cased"    
},    
"mbert-uncased": {        
"model_type": "bert",        
"model_name": "models--google-bert--bert-base-multilingual-uncased"    
},    
"roberta": {        
"model_type": "roberta",        
"model_name": "models--FacebookAI--roberta-base"    
},    
"beto-cased": {        
"model_type": "bert",        
"model_name": "models--dccuchile--bert-base-spanish-wwm-cased"    
},    
"beto-uncased": {        
"model_type": "bert",        
"model_name": "models--dccuchile--bert-base-spanish-wwm-uncased"    
},    
"distilbert-multi": {        
"model_type": "distilbert",        
"model_name": "models--distilbert--distilbert-base-multilingual-cased"    
}
}

In [8]:
from simpletransformers.classification import MultiLabelClassificationModel, MultiLabelClassificationArgs
for model, fields in models.items():    
  print("model:",model)
  models[model] = MultiLabelClassificationModel(fields["model_type"], os.path.join(TRANSFORMERS_PATH, model), use_cuda=False)

model: mbert-cased
model: mbert-uncased
model: roberta
model: beto-cased
model: beto-uncased
model: distilbert-multi


In [9]:
import json
best_ensemble = {}
with open(os.path.join(EXP_PATH, "best-ensemble.json")) as json_file:
    best_ensemble = json.load(json_file)
print("----- BEST ENSEMBLE -----")
for field in ["name", "models", "metrics"]:
  print(f"{field}:", best_ensemble.get(field))

----- BEST ENSEMBLE -----
name: ensemble18
models: ['beto-cased', 'beto-uncased']
metrics: {'accuracy': 0.97035, 'macro_f1': 0.98024, 'macro_precision': 0.99442, 'macro_recall': 0.96671, 'weighted_f1': 0.98177}


In [10]:
model_evaluation = {}
for model in best_ensemble.get("models"):  
  with open(os.path.join(os.path.join(EXP_PATH, model), "model-evaluation.json")) as json_file:
      model_evaluation[model] = json.load(json_file)

In [11]:
from sklearn.preprocessing import normalize
def vote(predictions, weighted=False, weights=None):
  votes = list()  
  for i in range(len(labels)):    
    curr_label_preds = [preds[i] for preds in predictions]    
    voting = sum(curr_label_preds * weights) if weighted else sum(curr_label_preds)/len(curr_label_preds)
    votes.append(0 if voting < 0.5 else 1)  
  return votes
test_predictions = list()
def predict_ensemble(ensemble_name, dataset_name, dataset, weighted=False):  
 for i in range(len(dataset.index)):
    predictions = list()
    ensemble_models = best_ensemble.get("models")    
    for model_name in ensemble_models:
      curr_model_outputs = model_evaluation[model_name].get(f"{dataset_name}_model_outputs")
      predictions.append(curr_model_outputs[i])
    weights = list()    
    if weighted:      
      f1_scores_list = [model_evaluation[model_name]["metrics"].get("weighted_f1")                        
                        for model_name in best_ensemble.get("models")]
      weights = normalize([f1_scores_list], norm="l1")[0]
    ensemble_pred = vote(predictions, weighted, weights)
    test_predictions.append(ensemble_pred)

In [12]:
for model_name in best_ensemble.get("models"):
  model_predictions, model_raw_outputs = models.get(model_name).predict(test["text"].tolist())
  model_evaluation[model_name]["test_model_outputs"] = model_raw_outputs
  model_evaluation[model_name]["test_predictions"] = model_predictions
predict_ensemble(best_ensemble.get("name"), "test", test, weighted=WEIGHTED)
n = 5
print(f"First {n} predictions:", test_predictions[:n])

3it [00:03,  1.32s/it]                                                                                                                                                                                                                  
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 186/186 [01:59<00:00,  1.55it/s]
3it [00:03,  1.13s/it]                                                                                                                                                                                                                  
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 186/186 [01:58<00:00,  1.57it/s]


First 5 predictions: [[0, 0, 0, 0, 1], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]


In [13]:
test_output = test[["id"]].rename(columns={"id": "id"})
for i in range(len(labels)):
  test_output[labels[i]] = [pred[i] for pred in test_predictions]
test_output.head(10)

,id,political,racial/ethnic,religious,gender/sexual,other
0,spa_1927fe24b83567b6383c90c35473cc32,0,0,0,0,1
1,spa_73b413c77432d415f2b63397ccbcda28,0,0,0,0,0
2,spa_5f0a632b6b0b4544b5fed6c84bd1b997,0,0,0,0,0
3,spa_b23cc0db2a4e552a93fbbfe4e4d2f45f,0,0,0,0,0
4,spa_90240d10f79a2761cbea0b76e1553a6f,0,0,0,0,0
5,spa_bfb5305232c58a214ace4bf4b74a0d08,0,0,0,0,0
6,spa_5734f86d92c6fbd89d4c2dae640bdf1f,1,0,0,0,1
7,spa_b16cfe3434da104f2f19105191b8ffbb,0,0,0,0,0
8,spa_c1ea290e5e71ec4a3666563cc62cf410,0,0,0,0,0
9,spa_d086bcb58ad688925bc703c8602e4117,0,0,0,0,0


In [14]:
test_output.to_csv(os.path.join(OUTPUT, "results_test3.csv"), index=False)